In [353]:
import pandas as pd
from urllib.request import Request, urlopen, urlretrieve
from pathlib import Path  
import ssl
import tqdm
import os
import re
from sklearn.preprocessing import MultiLabelBinarizer
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [152]:
BUCKET_NAME = 'art-api-sg'
#BUCKET_TRAIN_DATA_PATH = 'data/train_1k.csv'
BUCKET_TRAIN_DATA_PATH = 'yourpaintings'
PATH = Path("../raw_data")
PATH_YOURPAINTINGS = PATH/"yourpaintings"
MODEL_NAME = 'art-api'
MODEL_VERSION = 'baseline'

In [345]:
df_yourpaintings = pd.read_csv("https://www.robots.ox.ac.uk/~vgg/data/paintings/painting_dataset_2021.csv")
df_yourpaintings = df_yourpaintings.drop_duplicates()
df_yourpaintings = df_yourpaintings.dropna()
df_yourpaintings = df_yourpaintings[15:20]
df_yourpaintings

,Image URL,Web page URL,Subset,Labels
15,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-seventee...,'test',' chair diningtable dog'
16,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-20-havoc...,'test',' aeroplane'
17,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-basket-o...,'train',' bird dog'
18,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-basket-o...,'train',' bird'
19,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-bather-7936,'test',' chair'


In [346]:
test = pd.DataFrame([{'Image URL': 'http:// google.com',
                      'Web page URL': 'http://test.com',
                      'Subset': 'subset values',
                      'Labels': 'chair diningtable dog'}])

df_yourpaintings = pd.concat([df_yourpaintings, test], axis=0, ignore_index=True)
df_yourpaintings

,Image URL,Web page URL,Subset,Labels
0,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-seventee...,'test',' chair diningtable dog'
1,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-20-havoc...,'test',' aeroplane'
2,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-basket-o...,'train',' bird dog'
3,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-basket-o...,'train',' bird'
4,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-bather-7936,'test',' chair'
5,http:// google.com,http://test.com,subset values,chair diningtable dog


In [347]:
# for every row, save images from working urls and drop rows with inaccessible images
for index, row in df_yourpaintings.iterrows():
    url = str(row["Image URL"])
    fname = url[url.rfind("/")+1:]
    try:
        save_file = os.path.join(PATH_YOURPAINTINGS, fname)
        urlretrieve(url, save_file)
        df_yourpaintings.at[index, "filename"] = fname
    except:
        df_yourpaintings.drop(index=index, axis=0, inplace=True)

In [348]:
df_yourpaintings

,Image URL,Web page URL,Subset,Labels,filename
0,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-seventee...,'test',' chair diningtable dog',NY_YAG_YORAG_326-001.jpg
1,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-20-havoc...,'test',' aeroplane',LNE_RAFM_FA03538-001.jpg
2,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-basket-o...,'train',' bird dog',DUR_DBM_770-001.jpg
3,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-basket-o...,'train',' bird',DUR_DBM_769-001.jpg
4,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-bather-7936,'test',' chair',NY_YAG_YORAG_66-001.jpg


In [349]:
# extract labels into a new column called 'label_classes'
for index, row in df_yourpaintings.iterrows():
    text = str(row["Labels"])
    words_list = re.findall("[a-z]+", text, flags=re.IGNORECASE)
    df_yourpaintings.loc[index, "label_classes"] = 'test'
    df_yourpaintings.at[index, "label_classes"] = words_list

In [350]:
df_yourpaintings

,Image URL,Web page URL,Subset,Labels,filename,label_classes
0,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-seventee...,'test',' chair diningtable dog',NY_YAG_YORAG_326-001.jpg,"[chair, diningtable, dog]"
1,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-20-havoc...,'test',' aeroplane',LNE_RAFM_FA03538-001.jpg,[aeroplane]
2,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-basket-o...,'train',' bird dog',DUR_DBM_770-001.jpg,"[bird, dog]"
3,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-basket-o...,'train',' bird',DUR_DBM_769-001.jpg,[bird]
4,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-bather-7936,'test',' chair',NY_YAG_YORAG_66-001.jpg,[chair]


In [386]:
#save images that are successfuly downloaded into a clean csv
filepath = Path(yourpaintings_path/"df_yourpaintings.csv")  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_yourpaintings.to_csv(filepath, index=False)

In [389]:
df = pd.read_csv(filepath)
df

,Image URL,Web page URL,Subset,Labels,filename,label_classes
0,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-seventee...,'test',' chair diningtable dog',NY_YAG_YORAG_326-001.jpg,"['chair', 'diningtable', 'dog']"
1,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-20-havoc...,'test',' aeroplane',LNE_RAFM_FA03538-001.jpg,['aeroplane']
2,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-basket-o...,'train',' bird dog',DUR_DBM_770-001.jpg,"['bird', 'dog']"
3,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-basket-o...,'train',' bird',DUR_DBM_769-001.jpg,['bird']
4,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/a-bather-7936,'test',' chair',NY_YAG_YORAG_66-001.jpg,['chair']


In [ ]:
df["label_classes"] = df["label_classes"].astype("string")

In [388]:
#df['id'] = df['label_classes'].astype("string")

df['label_classes'].dtype

string[python]

# Build preprocessing pipeline to process images and encode in parallel

In [393]:
# mlb = MultiLabelBinarizer()

# test = df['label_classes']

# res = pd.DataFrame(mlb.fit_transform(df["label_classes"]),
#                    columns=mlb.classes_,
#                    index=test.index)

# res

# mlb = MultiLabelBinarizer(sparse_output=True)

# df = df.join(
#             pd.DataFrame.sparse.from_spmatrix(
#                 mlb.fit_transform(df["label_classes"]),
#                 index=df.index,
#                 columns=mlb.classes_))

mlb = MultiLabelBinarizer()

test = df['label_classes']

res = pd.DataFrame(mlb.fit_transform(test),
                   columns=mlb.classes_,
                   index=test.index)

res.shape

(5, 19)

In [392]:
res

,,',",",[,],a,b,c,d,e,g,h,i,l,n,o,p,r,t
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1
1,0,1,0,1,1,1,0,0,0,1,0,0,0,1,1,1,1,1,0
2,1,1,1,1,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0
3,0,1,0,1,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
4,0,1,0,1,1,1,0,1,0,0,0,1,1,0,0,0,0,1,0


In [ ]:
test = pd.Series([['a', 'b', 'e'], ['c', 'a'], ['d'], ['d'], ['e']])

mlb = MultiLabelBinarizer()

res = pd.DataFrame(mlb.fit_transform(test),
                   columns=mlb.classes_,
                   index=test.index)

In [5]:
def get_data():
    """method to get the training data (or a portion of it) from google cloud bucket"""
    df = pd.read_csv(f"gs://{BUCKET_NAME}/{BUCKET_TRAIN_DATA_PATH}", nrows=1000)
    return df

In [ ]:
def download(path, urls):
    # ignore SSL errors
    ssl._create_default_https_context = ssl._create_unverified_context
    for url in tqdm(urls):
        try:
            fname = url[url.rfind('/')+1:]
            downloaded_urls_lst.append(url)
            urllib.request.urlretrieve(url, path/fname)
        except Exception as e:
            logging.error(traceback.format_exc())
            
download(yourpaintings_path, urls)